# Dataframes and scikit-learn

In [ ]:
# this cell's metadata contains
# "nbsphinx": "hidden" so it is hidden by nbsphinx
PATH_YIELD_ENGINE = 'src'
def set_paths() -> None:
    import sys
    import os
    
    if 'cwd' not in globals():
        global cwd
        cwd = os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir)
        os.chdir(cwd)   
    print(f"working dir is '{os.getcwd()}'")                            
    if PATH_YIELD_ENGINE not in sys.path:
        sys.path.insert(0, PATH_YIELD_ENGINE)
    print(f"added `{sys.path[0]}` to python paths")
set_paths()

In [ ]:
import pandas as pd
# todo remove RandomForestRegressor when possible
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from pandas.api.types import is_numeric_dtype

from gamma import Sample
from gamma.viz.dendrogram import DendrogramDrawer, FeatMapStyle, LineStyle
from gamma.sklearndf.pipeline import PipelineDF
from gamma.sklearndf.transformation import SimpleImputerDF, MissingIndicatorDF, OneHotEncoderDF, ColumnTransformerDF
from gamma.sklearndf.transformation.extra import BorutaDF
from gamma.sklearndf.regression import RandomForestRegressorDF, LGBMRegressorDF
from gamma.sklearndf.classification import RandomForestClassifierDF
from gamma.model.inspection import ModelInspector
from gamma.model.prediction import PredictorFitCV
from gamma.model.selection import ModelPipelineDF, ModelGrid, ModelRanker, summary_report
from gamma.model.validation import CircularCrossValidator
from gamma.yieldengine.simulation import UnivariateSimulator
from gamma.yieldengine.partition import ContinuousRangePartitioning
from gamma.yieldengine.viz import SimulationDrawer, SimulationPlotStyle

In [ ]:
df = pd.read_csv('data/ames-housing-dataset/train.csv')
df = df.drop(['Id', 'YrSold', 'MoSold'], axis=1)
TARGET = "SalePrice"

In [ ]:
y = df[TARGET]

In [ ]:
df = df.drop(columns=TARGET)

## Transformers and Pipeline

Some features are categorical, like **GarageType** and there are some missing values:

In [ ]:
df.GarageType.unique()

In [ ]:
df.GarageType.isna().sum()

With scikit-learn to one-hot encode one should first impute missing values, and then one hot encode categorical variables:

In [ ]:
categorical_features = df.select_dtypes([object]).columns
numerical_features = [col for col, dtype in df.dtypes.iteritems() if is_numeric_dtype(dtype)]

### With sklearn

In [ ]:
ohe_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='nan')),
    ('ohe', OneHotEncoder(sparse=False))
])

In [ ]:
imputer = SimpleImputer(strategy="median")

In [ ]:
ohe_transformer = ColumnTransformer(
    transformers=[('categorical', ohe_pipeline, categorical_features), 
                  ('numerical', imputer, numerical_features)])

In [ ]:
array_transformed = ohe_transformer.fit_transform(df)

In [ ]:
array_transformed

In [ ]:
array_transformed.shape

### With sklearndf

In [ ]:
ohe_pipeline_df = PipelineDF(steps=[
    ('imputer', SimpleImputerDF(strategy='constant', fill_value='nan')),
    ('ohe', OneHotEncoderDF(sparse=False, handle_unknown='ignore'))
])

In [ ]:
imputer_df = SimpleImputerDF(strategy="median")

In [ ]:
ohe_transformer_df = ColumnTransformerDF(
    transformers=[('categorical', ohe_pipeline_df, categorical_features),
                  ('numerical', imputer_df, numerical_features)])

In [ ]:
transformed_df = ohe_transformer_df.fit_transform(df)

In [ ]:
transformed_df.head()

In [ ]:
ohe_transformer_df.columns_original.head()

## Regressor

TODO

In [ ]:
random_forest_regressor_df = RandomForestRegressorDF(n_estimators=50)

In [ ]:
df_numerical = df.select_dtypes(include='number').fillna(0)
df_numerical_train, df_numerical_test, y_train, y_test = train_test_split(df_numerical, y)

In [ ]:
random_forest_regressor_df.fit(df_numerical_train, y_train)

In [ ]:
random_forest_regressor_df.score(df_numerical_test, y_test)

In [ ]:
random_forest_regressor_df.get_params()

In [ ]:
random_forest_regressor_df.set_params(max_depth=10)

In [ ]:
random_forest_regressor_df.delegate_estimator

In [ ]:
random_forest_regressor_df.columns_in

In [ ]:
random_forest_regressor_df.is_fitted

## Classifier

TODO

In [ ]:
y_bin = y.apply(lambda x: 0 if x < y.median() else 1)

In [ ]:
random_forest_classifier_df = RandomForestClassifierDF(n_estimators=50)

In [ ]:
random_forest_classifier_df.fit(df_numerical, y_bin)

In [ ]:
random_forest_classifier_df.score(df_numerical, y_bin)

In [ ]:
random_forest_classifier_df.get_params()

In [ ]:
random_forest_classifier_df.set_params(max_depth=10)

In [ ]:
random_forest_classifier_df.delegate_estimator

In [ ]:
random_forest_classifier_df.columns_in

In [ ]:
random_forest_classifier_df.is_fitted

## Pipeline

TODO

In [ ]:
ohe_pipeline_df = PipelineDF(steps=[
    ('imputer', SimpleImputerDF(strategy='constant', fill_value='nan')),
    ('ohe', OneHotEncoderDF(sparse=False, handle_unknown='ignore'))
])

In [ ]:
imputer_df = SimpleImputerDF(strategy="median")

In [ ]:
ohe_transformer_df = ColumnTransformerDF(
    transformers=[('categorical', ohe_pipeline_df, categorical_features),
                  ('numerical', imputer_df, numerical_features)])

In [ ]:
full_pipeline_df = PipelineDF(steps=[
    ('preprocessing', ohe_transformer_df),
    ('rf_model', RandomForestRegressorDF(n_estimators=10))
])

In [ ]:
df_train, df_test, y_train, y_test = train_test_split(df, y)

In [ ]:
ohe_transformer_df.fit(df_train)

In [ ]:
ohe_transformer_df.transform(df_test).head()

In [ ]:
full_pipeline_df.fit(df_train, y_train)

In [ ]:
full_pipeline_df.predict(df_test).head()

In [ ]:
full_pipeline_df.score(df_test, y_test)

## Extra

TODO
Boruta, LGBM...